# Operaciones con Ventanas y Joins en Spark

Spark permite aplicar funciones de ventana y realizar joins entre DataFrames, herramientas esenciales para análisis complejos como rankings, cálculos acumulados y combinaciones de datasets.

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, avg
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

# Crear la sesión de Spark
spark = SparkSession.builder.appName("PySpark04").getOrCreate()

# Dataset de ejemplo
data = [
    ("Alice", "HR", 3000),
    ("Bob", "IT", 4500),
    ("Cathy", "HR", 3200),
    ("David", "IT", 5000),
    ("Eve", "Finance", 4000),
    ("Frank", "Finance", 4200)
]
columns = ["name", "department", "salary"]

df = spark.createDataFrame(data, columns)
df.show()

## Operaciones de Ventana

Las Window functions permiten realizar cálculos sobre un grupo de filas relacionadas, pero sin colapsarlas en una sola fila (a diferencia de groupBy). Muy útil para rankings, medias móviles, diferencias entre filas, etc.

#### Componentes de una Window
- *partitionBy*: cómo dividir los datos (ej. por usuario)
- *orderBy*: cómo ordenar cada partición (ej. por valoraciones)

### Rankear empleados por salario dentro de su departamento

In [ ]:
window_spec = Window.partitionBy("department").orderBy(col("salary").desc())

df_ranked = df.withColumn("rank", row_number().over(window_spec))
df_ranked.show()

### Calcular salario promedio por departamento con función de ventana

In [ ]:
df_avg = df.withColumn("avg_salary", avg("salary").over(Window.partitionBy("department")))
df_avg.show()

## Joins entre DataFrames

Los joins permiten combinar datos de dos DataFrames en función de columnas comunes (que se recomienda que sean ids). Es fundamental para enriquecer un conjunto de datos con información de otro.

|Tipo|Descripción|
|----------|----------|
|inner|Devuelve solo las filas que coinciden en ambas tablas.|
|left|Devuelve todas las filas de la izquierda y las coincidencias de la derecha. Si no hay coincidencia, los valores de la derecha son null.|
|right|Igual que left, pero para el DataFrame derecho.|
|outer|Devuelve todas las filas, con null donde no hay coincidencia.|
|semi|Devuelve solo las filas del DataFrame izquierdo que tienen coincidencia, sin añadir columnas del derecho.|
|anti|Devuelve las filas del DataFrame izquierdo que no tienen coincidencia en el derecho.|

### Dataset adicional: managers

In [ ]:
managers = [("HR", "Laura"), ("IT", "Steve"), ("Finance", "Paul")]
mgr_columns = ["department", "manager"]

df_mgr = spark.createDataFrame(managers, mgr_columns)
df_mgr.show()

### Join de empleados con sus managers

In [ ]:
df_joined = df.join(df_mgr, on="department", how="inner")
df_joined.select("name", "department", "salary", "manager").show()

## Ejercicio
1. Carga los datos de */../data/movies.csv* y los de *../data/ratings.csv* que contienen datos de películas y valoraciones, respectivamente. 
2. Añade los títulos de las películas a los datos de puntuaciones
3. Extrae la película más valorada por cada usuario usando una función ventana

In [ ]:
movies = spark.read.csv("../../data/movies.csv", header=True, inferSchema=True)
ratings = spark.read.csv("../../data/ratings.csv", header=True, inferSchema=True)

In [ ]:
# Añade los títulos de las películas a los datos de puntuaciones
ratings_with_titles = ratings.join(movies.select("movieId", "title"), on="movieId", how="inner")
ratings_with_titles.show(10)

In [ ]:
#Película más valorada por cada usuario
window_user = Window.partitionBy("userId").orderBy(ratings_with_titles["rating"].desc())

top_movies_per_user = ratings_with_titles \
    .withColumn("rank", f.row_number().over(window_user)) \
    .filter("rank = 1")

print('Película más valorada por cada usuario:')
top_movies_per_user.select("userId", "title", "rating").show(10)

In [ ]:
spark.stop()